In [333]:
import glob
import cv2
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
import shutil

In [4]:
#Nothing to see here

In [58]:
class generate:
    def batch_1(img):
        """ Requires uint8 image dtype"""
        images_arrays = []
        images_arrays.append(generate.rotation(img, -90))
        images_arrays.append(generate.rotation(img, 90))
        images_arrays.append( generate.vertical_flip(img))
        images_arrays.append(generate.high_brightness(img))
        images_arrays.append(generate.low_brightness(img))
        images_arrays.append( generate.sharpen(img))

        return images_arrays

    
    def rotation(image,angle):
        height, width = image.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, scale=1)
        rotated_image = cv2.warpAffine(image,rotation_matrix,(width,height))
        return rotated_image

 
    def vertical_flip(image):
        return cv2.flip(image,0)
    
    def high_brightness(image):
        return cv2.add(image, np.ones(image.shape , dtype=np.uint8) * 40)

    def low_brightness(image):
        return cv2.subtract(image, np.ones(image.shape , dtype=np.uint8) * 40)
        

    def sharpen(image):
        kernel = np.array([ [-1,-1,-1],
                            [-1,10,-1],
                            [-1,-1,-1] ])
        return cv2.filter2D(image,-1,kernel)

In [276]:
def fetch_imgs_path(path, class_encoding, equal_ratio_to_healthy, shuffle):
    class_folders = os.listdir(path)
    imgs_path_dict = {}
    for class_folder in class_folders:
        files = [path + class_folder + '/' + file for file in os.listdir(path + class_folder + '/')]
        if shuffle == True:
            random.shuffle(files)
        imgs_path_dict[str(class_folder)] = files
    if class_encoding == 'binary':
        temp_dict = {'unhealthy':[], 'healthy':[]}
        if equal_ratio_to_healthy == True:
            ratio = int(len(imgs_path_dict['healthy']) / (len(class_folders) - 1))
            for folder, img_files  in imgs_path_dict.items():
                if folder == 'healthy':
                    temp_dict['healthy'] = random.sample(imgs_path_dict[folder], len(imgs_path_dict['healthy']))
                else:
                    temp_dict['unhealthy'] = temp_dict['unhealthy'] + random.sample(imgs_path_dict[folder], ratio)
        
            imgs_path_dict = temp_dict   
            
        else:
            for folder, img_files  in imgs_path_dict.items():
                if folder == 'healthy':
                    temp_dict['healthy'] = imgs_path_dict[folder]
                else:
                    temp_dict['unhealthy'] = temp_dict['unhealthy'] + imgs_path_dict[folder]
            
            imgs_path_dict = temp_dict
            
        
    return imgs_path_dict

In [436]:
b = [3,4,5,7,6]

In [433]:
b

[3, 4, 5, 7, 6]

In [444]:
b = [3,4,5,7,6]
for item in range(len(b)):
    print(len(b))
    b.pop()   

5
4
3
2
1


In [601]:
def save_generation(img_files, split, folder_path, class_folder, names, augment=1):
    for img_file in img_files:
        img = cv2.imread(img_file)
        cv2.imwrite(folder_path + class_folder + '/' + str(names.pop()) + '.jpg', img)
        if augment > 0:
            if augment == 1:
                img_array = generate.batch_1(img)
                for generic_img in img_array:
                    cv2.imwrite(folder_path + split + class_folder + '/' + str(names.pop()) + '.jpg', generic_img)
        del img_file

In [623]:
def visualise_tree(path):
    tree = {}
    for split in os.listdir(path):
        tree[split] = {}
        for class_folder in os.listdir(path + split + '/'):
            imgs = os.listdir(path + split + '/' + class_folder + '/')
            tree[split][class_folder] = {}
            tree[split][class_folder]['original'] = int(len(imgs)/7) + 1
            tree[split][class_folder]['generic'] = int(len(imgs)/7*6) - 1
    
    for split_folder, split_file in visualise_tree('temp/').items():
    print(split_folder, 'folder')
    for class_folder, class_files in split_file.items():
        print(f'    |--class {class_folder}')
        for ratio_id, ratio in  class_files.items():
            print(f'          |--contains {ratio} {ratio_id} images')
    return tree

test folder
    |--class healthy
          |--contains 1 original images
          |--contains 4 generic images
    |--class unhealthy
          |--contains 1 original images
          |--contains 4 generic images
train folder
    |--class healthy
          |--contains 697 original images
          |--contains 4180 generic images
    |--class unhealthy
          |--contains 1815 original images
          |--contains 10886 generic images
val folder
    |--class healthy
          |--contains 199 original images
          |--contains 1192 generic images
    |--class unhealthy
          |--contains 519 original images
          |--contains 3110 generic images


In [606]:
def to_split_folder(folder_path, img_path_dict, ratio, augment):
    folder_splits = ('train/', 'val/', 'test/')
    try:
        shutil.rmtree(folder_path)
    except FileNotFoundError:
        pass
    os.mkdir(folder_path)
        
    for index, split in enumerate(ratio):
        if sum(ratio) > 1:
            print('Split Ratio is Invalid!')
            break
        os.mkdir(folder_path + folder_splits[index])
        for folder in img_path_dict.keys():
            os.mkdir(folder_path + folder_splits[ratio.index(split)] + folder)
        if ratio.index(split) == 0:
            for class_folder, img_files in img_path_dict.items():
                img_files = img_files[:int(len(img_files)*split)]  
                names = [id for id in range((len(img_files)) * 7)]
                random.shuffle(names)
                save_generation(img_files=img_files, split='train/', folder_path=folder_path, class_folder=class_folder, names=names)
        if ratio.index(split) == 1:
            for class_folder, img_files in img_path_dict.items():
                img_files = img_files[int(len(img_files)*ratio[0]):(int(len(img_files)*ratio[0])+int(len(img_files)*split))]                
                names = [id for id in range((len(img_files)) * 7)]
                random.shuffle(names)
                save_generation(img_files=img_files, split='val/', folder_path=folder_path, class_folder=class_folder, names=names)
        if ratio.index(split) == 2:
            for class_folder, img_files in img_path_dict.items():
                img_files = img_files[(int(len(img_files)*ratio[0])+(int(len(img_files)*ratio[1]))):]
                names = [id for id in range((len(img_files)) * 7)]
                random.shuffle(names)
                save_generation(img_files=img_files, split='test/', folder_path=folder_path, class_folder=class_folder, names=names)

In [598]:
class compile:
    path = 'src/'
    temp_file = 'temp/'
    def __init__(self, class_encoding, ratio=[0.7,.2,.1], equal_ratio_to_healthy=False, shuffle=True, augment=1, random_file_name=True):
        self.dataset_dir = fetch_imgs_path(self.path, class_encoding, equal_ratio_to_healthy, shuffle)
        #to split_folders
        to_split_folder(folder_path=self.temp_file, img_path_dict=self.dataset_dir, ratio=ratio, augment=augment)
    
    @property
    def load(self):
        return None

In [599]:
b = compile(class_encoding='binary', ratio=[.7,.2,.1], equal_ratio_to_healthy=False, shuffle=True, augment=1, random_file_name=True)

In [572]:
for keys, items  in b.dataset_dir.items():
    print(keys, len(items))

unhealthy 3025
healthy 1162


In [603]:
int(1392/7)

198.85714285714286